In [54]:
import numpy as np
import pandas as pd
import sys
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
from generating_feature import generating_pro_feature, generating_drug_feature, concat_feature
sys.path = original_path

In [55]:
# Drug feature extracting

file_path = "dti2vec_drugbankid2smiles"  # Specify the file path without the extension

# Read the file as a CSV with custom parameters

df = pd.read_csv(file_path, header=None, delimiter="\t")  # Assuming the file has no header row
print(len(df))
print(df.iloc[[0,1,300,600, 1000]])
df.columns = ["Drug_ID", "Drug"]

9881
            0                                                  1
0     DB00672                 CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl
1     DB00116  C1C(NC2=C(N1)N=C(NC2=O)N)CNC3=CC=C(C=C3)C(=O)N...
300   DB00419          CCCCN1C[C@@H]([C@H]([C@@H]([C@H]1CO)O)O)O
600   DB00714       CN1CCC2=C3[C@H]1CC4=C(C3=CC=C2)C(=C(C=C4)O)O
1000  DB00497  CN1CC[C@]23[C@@H]4C(=O)CC[C@]2([C@H]1CC5=C3C(=...


In [91]:
import pickle 

with open('VEC_drug_feats.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
x

{'Drug_ID': array(['DB00672', 'DB06663', 'DB00586', ..., 'DB00610', 'DB08903',
        'DB00474'], dtype=object),
 'Drug': array(['CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl',
        'C1[C@H](CN2[C@@H]1C(=O)N[C@H](C(=O)N[C@@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C2=O)CC3=CC=CC=C3)CC4=CC=C(C=C4)OCC5=CC=CC=C5)CCCCN)CC6=CNC7=CC=CC=C76)C8=CC=CC=C8)OC(=O)NCCN',
        'C1=CC=C(C(=C1)CC(=O)O)NC2=C(C=CC=C2Cl)Cl', ...,
        'C[C@@H]([C@@H](C1=CC(=CC=C1)O)O)N',
        'CN(C)CC[C@@](C1=CC=CC2=CC=CC=C21)([C@H](C3=CC=CC=C3)C4=C(N=C5C=CC(=CC5=C4)Br)OC)O',
        'CCC#CC(C)C1(C(=O)NC(=O)N(C1=O)C)CC=C'], dtype=object),
 'Morgan_Features': array([array([0.28844152, 0.03085759, 0.33230764, ..., 0.06655665, 0.05585421,
               0.00247426])                                                    ,
        array([0.05462653, 0.3620248 , 0.11030577, ..., 0.31672817, 0.16140309,
               0.17364596])                                                    ,
        array([0.13667219, 0.01604207, 0.0090496

In [56]:
# Load json file for mapping encoding & Drug ID

import pickle
import json

temp = []

# JSON 파일 경로 설정
json_file = 'node2id.json'

# JSON 파일 읽기
with open(json_file) as f:
    data = json.load(f)
    
for idx, item in enumerate(df["Drug_ID"].copy()) :

    for jdx, d_name in enumerate(data.keys()) :
        if d_name == item :
            temp.append(data[d_name])
            break
        elif jdx +1 == len(data)  :
            temp.append(np.NaN)


In [72]:
df["drug_enco"] = temp
df.dropna().drop_duplicates()

,Drug_ID,Drug,drug_enco
0,DB00672,CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl,569.0
2,DB06663,C1[C@H](CN2[C@@H]1C(=O)N[C@H](C(=O)N[C@@H](C(=...,1030.0
4,DB00586,C1=CC=C(C(=C1)CC(=O)O)NC2=C(C=CC=C2Cl)Cl,668.0
6,DB00461,CC(=O)CCC1=CC2=C(C=C1)C=C(C=C2)OC,804.0
7,DB01151,CNCCCN1C2=CC=CC=C2CCC3=CC=CC=C31,231.0
...,...,...,...
9640,DB00385,CCCCC(=O)OCC(=O)[C@]1(C[C@@H](C2=C(C1)C(=C3C(=...,895.0
9751,DB04865,CC(C)(CCC[C@@](CC(=O)OC)(C(=O)O[C@H]1[C@H]2C3=...,1188.0
9760,DB00610,C[C@@H]([C@@H](C1=CC(=CC=C1)O)O)N,728.0
9787,DB08903,CN(C)CC[C@@](C1=CC=CC2=CC=CC=C21)([C@H](C3=CC=...,1315.0


In [58]:
# Be alert to know there is few ways to represent Drug Feature 
# In this project we've chosen two type(morgan, map4)

# get Morgan fingerprint's Feature for Drug


# Select unique rows based on the 'Drug' column
unique_drugs_df = df.drop_duplicates(subset=['Drug']).dropna()

unique_drugs_df = generating_drug_feature(unique_drugs_df, 256)
# Print the resulting DataFrame
# print(len(unique_drugs_df))
# print(unique_drugs_df)
tmp = unique_drugs_df[['Drug_ID', 'Drug','Morgan_Features', 'drug_enco']]
# print(tmp.head())

drug_id_array = tmp['Drug_ID'].values
drug_array = tmp['Drug'].values
drug_feat = tmp['Morgan_Features']
drug_enco = tmp['drug_enco'].values



drug_feat = {'Drug_ID': drug_id_array, 'Drug': drug_array, 'Morgan_Features': drug_feat, "Drug_enco" : drug_enco}

drug_feat["Morgan_Features"] = drug_feat["Morgan_Features"].apply(lambda x : [int(y) for y in x])


import pickle


Dictionary saved as pickle file


In [79]:
# get MAP4 Features

import pandas as pd
from rdkit import Chem
import tmap as tm
from map4 import MAP4Calculator
from sklearn.preprocessing import MinMaxScaler

dim = 1024

MAP4 = MAP4Calculator(dimensions=dim)
ENC = tm.Minhash(dim)

map4_arr = []
scaler = MinMaxScaler()

#smiles = pd.read_csv("dti2vec_drugbankid2smiles.txt", delimiter="\t", header=None)
map4_df = df.dropna().drop_duplicates()
for i in map4_df["Drug"] :
    smiles_b = i
    mol_b = Chem.MolFromSmiles(smiles_b)
    map4_b = MAP4.calculate(mol_b)
    scaled_data = scaler.fit_transform(np.array(map4_b).reshape(-1,1))
    result = scaled_data.reshape(1,-1)[0]
    map4_arr.append(result)

map4_df["MAP4"] = map4_arr
map4_df
    

,Drug_ID,Drug,drug_enco,MAP4
0,DB00672,CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl,569.0,"[0.2884415215905463, 0.03085758668186381, 0.33..."
2,DB06663,C1[C@H](CN2[C@@H]1C(=O)N[C@H](C(=O)N[C@@H](C(=...,1030.0,"[0.054626531602622715, 0.3620248040210776, 0.1..."
4,DB00586,C1=CC=C(C(=C1)CC(=O)O)NC2=C(C=CC=C2Cl)Cl,668.0,"[0.13667219045090595, 0.01604206967039705, 0.0..."
6,DB00461,CC(=O)CCC1=CC2=C(C=C1)C=C(C=C2)OC,804.0,"[0.09919646710129056, 0.10284225401201255, 0.0..."
7,DB01151,CNCCCN1C2=CC=CC=C2CCC3=CC=CC=C31,231.0,"[0.09418569452788335, 0.12503912080866142, 0.0..."
...,...,...,...,...
9640,DB00385,CCCCC(=O)OCC(=O)[C@]1(C[C@@H](C2=C(C1)C(=C3C(=...,895.0,"[0.12066004881314797, 0.04249036062622785, 0.0..."
9751,DB04865,CC(C)(CCC[C@@](CC(=O)OC)(C(=O)O[C@H]1[C@H]2C3=...,1188.0,"[0.16401754794091294, 0.1422290992590266, 0.03..."
9760,DB00610,C[C@@H]([C@@H](C1=CC(=CC=C1)O)O)N,728.0,"[0.22865870541438305, 0.0650083039681738, 0.06..."
9787,DB08903,CN(C)CC[C@@](C1=CC=CC2=CC=CC=C21)([C@H](C3=CC=...,1315.0,"[0.02194528594788913, 0.03820902761785325, 0.0..."


In [105]:
map4_df['MAP4'].apply(lambda x : [str(y) for y in x])

0       [0.2884415215905463, 0.03085758668186381, 0.33...
2       [0.054626531602622715, 0.3620248040210776, 0.1...
4       [0.13667219045090595, 0.01604206967039705, 0.0...
6       [0.09919646710129056, 0.10284225401201255, 0.0...
7       [0.09418569452788335, 0.12503912080866142, 0.0...
                              ...                        
9640    [0.12066004881314797, 0.04249036062622785, 0.0...
9751    [0.16401754794091294, 0.1422290992590266, 0.03...
9760    [0.22865870541438305, 0.0650083039681738, 0.06...
9787    [0.02194528594788913, 0.03820902761785325, 0.0...
9801    [0.05973067658107211, 0.006250027319315053, 0....
Name: MAP4, Length: 1331, dtype: object

In [89]:
drug_id_array = map4_df['Drug_ID'].values
drug_array = map4_df['Drug'].values
drug_feat = map4_df['MAP4'].values
drug_enco = map4_df['drug_enco'].values


drug_feat = {'Drug_ID': drug_id_array, 'Drug': drug_array, 'Morgan_Features': drug_feat, "Drug_enco" : drug_enco}

drug_feat



{'Drug_ID': array(['DB00672', 'DB06663', 'DB00586', ..., 'DB00610', 'DB08903',
        'DB00474'], dtype=object),
 'Drug': array(['CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl',
        'C1[C@H](CN2[C@@H]1C(=O)N[C@H](C(=O)N[C@@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C2=O)CC3=CC=CC=C3)CC4=CC=C(C=C4)OCC5=CC=CC=C5)CCCCN)CC6=CNC7=CC=CC=C76)C8=CC=CC=C8)OC(=O)NCCN',
        'C1=CC=C(C(=C1)CC(=O)O)NC2=C(C=CC=C2Cl)Cl', ...,
        'C[C@@H]([C@@H](C1=CC(=CC=C1)O)O)N',
        'CN(C)CC[C@@](C1=CC=CC2=CC=CC=C21)([C@H](C3=CC=CC=C3)C4=C(N=C5C=CC(=CC5=C4)Br)OC)O',
        'CCC#CC(C)C1(C(=O)NC(=O)N(C1=O)C)CC=C'], dtype=object),
 'Morgan_Features': array([array([0.28844152, 0.03085759, 0.33230764, ..., 0.06655665, 0.05585421,
               0.00247426])                                                    ,
        array([0.05462653, 0.3620248 , 0.11030577, ..., 0.31672817, 0.16140309,
               0.17364596])                                                    ,
        array([0.13667219, 0.01604207, 0.0090496

In [101]:
map4_df[["drug_enco", "MAP4"]]

,drug_enco,MAP4
0,569.0,"[0.2884415215905463, 0.03085758668186381, 0.33..."
2,1030.0,"[0.054626531602622715, 0.3620248040210776, 0.1..."
4,668.0,"[0.13667219045090595, 0.01604206967039705, 0.0..."
6,804.0,"[0.09919646710129056, 0.10284225401201255, 0.0..."
7,231.0,"[0.09418569452788335, 0.12503912080866142, 0.0..."
...,...,...
9640,895.0,"[0.12066004881314797, 0.04249036062622785, 0.0..."
9751,1188.0,"[0.16401754794091294, 0.1422290992590266, 0.03..."
9760,728.0,"[0.22865870541438305, 0.0650083039681738, 0.06..."
9787,1315.0,"[0.02194528594788913, 0.03820902761785325, 0.0..."


In [90]:
# save as pkl file

import pickle
with open("VEC_drug_feats.pkl", 'wb') as f:
    pickle.dump(drug_feat, f)

print(f"Dictionary saved as pickle file")

Dictionary saved as pickle file


In [53]:
len(drug_feat["Drug_ID"])

1331

In [ ]:
# Target Feature Extracting

In [5]:
'''df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
df[:2]'''

,Target_ID,Target
0,P28335,MVNLRNAVHSFLVHLIGLLVWQSDISVSPVAAIVTDIFNTSDGGRF...
1,P14416,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...


In [ ]:
# Extract Target Feature using ProtBert 

df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','ProtBERT_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['ProtBERT_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'ProtBERT_Features': target_feat}

#print(len(target_feat['ProtBERT_Features']))
#print(type(target_feat['ProtBERT_Features']))
#print(target_feat)

#import pickle


#with open("VEC_target_feats_tmp.pkl", 'wb') as f:
#    pickle.dump(target_feat, f)

#print(f"Dictionary saved as pickle file")


1407


Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


debug_point ['...', 'MAATGANAEKAESHKSGARHCGHNRAGSGYLLQGRV', '...']


In [13]:
target_feat[""]

{'Target_ID': array(['A5X5Y0', 'Q03013', 'Q04844', ..., 'P18031', 'P17948', 'Q9Y6R1'],
       dtype=object),
 'Target': array(['MLAFILSRATPRPALGPLSYREHRVALLHLTHSMSTTGRGVTFTINCSGFGQHGADPTALNSVFNRKPFRPVTNISVPTQVNISFAMSAILDVNEQLHLLSSFLWLEMVWDNPFISWNPEECEGITKMSMAAKNLWLPDIFIIELMDVDKTPKGLTAYVSNEGRIRYKKPMKVDSICNLDIFYFPFDQQNCTLTFSSFLYTVDSMLLDMEKEVWEITDASRNILQTHGEWELLGLSKATAKLSRGGNLYDQIVFYVAIRRRPSLYVINLLVPSGFLVAIDALSFYLPVKSGNRVPFKITLLLGYNVFLLMMSDLLPTSGTPLIGVYFALCLSLMVGSLLETIFITHLLHVATTQPPPLPRWLHSLLLHCNSPGRCCPTAPQKENKGPGLTPTHLPGVKEPEVSAGQMPGPAEAELTGGSEWTRAQREHEAQKQHSVELWLQFSHAMDAMLFRLYLLFMASSIITVICLWNT',
        'MSMTLGYWDIRGLAHAIRLLLEYTDSSYEEKKYTMGDAPDYDRSQWLNEKFKLGLDFPNLPYLIDGAHKITQSNAILCYIARKHNLCGETEEEKIRVDILENQAMDVSNQLARVCYSPDFEKLKPEYLEELPTMMQHFSQFLGKRPWFVGDKITFVDFLAYDVLDLHRIFEPNCLDAFPNLKDFISRFEVSCGIM',
        'MARAPLGVLLLLGLLGRGVGKNEELRLYHHLFNNYDPGSRPVREPEDTVTISLKVTLTNLISLNEKEETLTTSVWIGIDWQDYRLNYSKDDFGGIETLRVPSELVWLPEIVLENNIDGQFGVAYDANVLVYEGGSVTWLPPAIYRSVCAVEVTYFPFDWQNCSLIFRSQTYNAEEVEFTFAV

In [14]:
# load json file for mapping Target ID & Encoding number

with open('entity_drug.json', 'r') as f:
    data = json.load(f)

df = pd.json_normalize(data)

df = df.T
df

,0
Gene::801,1710
Gene::7428,1711
Gene::5987,1712
Gene::9412,1713
Gene::5747,1714
...,...
Gene::255349,34119
Gene::387104,34120
Gene::347541,34121
Gene::4114,34122


In [16]:
target_feat["Target_ID"]

import pandas as pd
a = pd.read_csv("convert_table", delimiter= "\t")

array(['A5X5Y0', 'Q03013', 'Q04844', ..., 'P18031', 'P17948', 'Q9Y6R1'],
      dtype=object)

In [17]:
# change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID)

import pandas as pd
a = pd.read_csv("convert_table", delimiter= "\t")

new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["ProtBERT_Features"][x_idx])
            break


In [18]:
# store as DataFrame
new_x = {}
new_x["Target_ID"] = new
new_x["ProtBERT_Features"]= new_emb
new_x = pd.DataFrame(new_x)
new_x

         Target_ID                                  ProtBERT_Features
0     Gene::285242  [0.035330534, 0.020144427, 0.08968215, -3.7893...
1       Gene::2948  [0.08002438, 0.043003686, 0.13087691, -0.02666...
2       Gene::1145  [0.05192073, 0.018403832, 0.08980505, 0.028965...
3       Gene::1645  [0.08332656, 0.031692903, 0.17612904, 0.029655...
4         Gene::90  [0.055903114, 0.03786295, 0.14410946, 0.015372...
...            ...                                                ...
1380    Gene::6715  [0.036394518, 0.010600796, 0.13814189, 0.02396...
1381     Gene::151  [0.029553, -0.001530874, 0.08643232, 0.0308537...
1382    Gene::5770  [0.05979089, -0.0039166478, 0.092263274, 0.011...
1383    Gene::2321  [0.09563119, -0.0071882573, 0.08168799, -0.000...
1384    Gene::8671  [0.10289173, -0.01595681, 0.09723776, 0.041603...

[1385 rows x 2 columns]


In [21]:
# Convert Gene ID to Encoding number And Build Mapping array for training

import json

with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))

In [27]:
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "ProtBERT_Features" : new_x["ProtBERT_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}
vec_pfeat

{'Target_ID': array(['Gene::1586', 'Gene::91', 'Gene::5291', ..., 'Gene::3782',
        'Gene::8510', 'Gene::6330'], dtype=object),
 'ProtBERT_Features': 0       [0.10895072, 0.07753971, 0.14122498, -0.004359...
 1       [0.07643488, 0.046613887, 0.11374605, 0.034627...
 2       [0.079430476, 0.028320394, 0.066061325, 0.0013...
 3       [0.13558188, 0.013845695, 0.12252106, 0.037885...
 4       [0.11597116, 0.016289648, 0.10930617, 0.023096...
                               ...                        
 1380    [0.026317561, 0.002467662, 0.088176705, 0.0322...
 1381    [0.025081605, 0.011302358, 0.13916573, 0.03344...
 1382    [0.06859535, -0.012777425, 0.09564611, 0.01458...
 1383    [0.05675855, -0.017336804, 0.099945456, 0.0534...
 1384    [0.08563626, -0.021550683, 0.13873681, 0.03547...
 Name: ProtBERT_Features, Length: 1385, dtype: object,
 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]),
 'map_arr': array([   0,    1,    2, ..., 1382, 1383, 1384])}

In [30]:
# save as pkl file

import pickle
with open("VEC_target_feats_tmp.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)

print(f"Dictionary saved as pickle file")

Dictionary saved as pickle file


In [93]:
# Check pickle file

with open('VEC_target_feats.pkl', 'rb') as f:
    a = pickle.load(f, encoding='utf-8')
print(a.keys())
print(type((a['ProtBERT_Features'])))
print(a)

dict_keys(['Target_ID', 'ProtBERT_Features', 'Gene_enco', 'map_arr'])
<class 'pandas.core.series.Series'>
{'Target_ID': array(['Gene::1586', 'Gene::91', 'Gene::5291', ..., 'Gene::3782',
       'Gene::8510', 'Gene::6330'], dtype=object), 'ProtBERT_Features': 0       [0.10895072, 0.07753971, 0.14122498, -0.004359...
1       [0.07643488, 0.046613887, 0.11374605, 0.034627...
2       [0.079430476, 0.028320394, 0.066061325, 0.0013...
3       [0.13558188, 0.013845695, 0.12252106, 0.037885...
4       [0.11597116, 0.016289648, 0.10930617, 0.023096...
                              ...                        
1380    [0.026317561, 0.002467662, 0.088176705, 0.0322...
1381    [0.025081605, 0.011302358, 0.13916573, 0.03344...
1382    [0.06859535, -0.012777425, 0.09564611, 0.01458...
1383    [0.05675855, -0.017336804, 0.099945456, 0.0534...
1384    [0.08563626, -0.021550683, 0.13873681, 0.03547...
Name: ProtBERT_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   na

0       [0.10895072, 0.07753971, 0.14122498, -0.004359...
1       [0.07643488, 0.046613887, 0.11374605, 0.034627...
2       [0.079430476, 0.028320394, 0.066061325, 0.0013...
3       [0.13558188, 0.013845695, 0.12252106, 0.037885...
4       [0.11597116, 0.016289648, 0.10930617, 0.023096...
                              ...                        
1380    [0.026317561, 0.002467662, 0.088176705, 0.0322...
1381    [0.025081605, 0.011302358, 0.13916573, 0.03344...
1382    [0.06859535, -0.012777425, 0.09564611, 0.01458...
1383    [0.05675855, -0.017336804, 0.099945456, 0.0534...
1384    [0.08563626, -0.021550683, 0.13873681, 0.03547...
Name: ProtBERT_Features, Length: 1385, dtype: object